# Imports

In [9]:
from dqn_agent import DQNAgent
from ppo_agent import PPOAgent
from ffnn_agent import FFNNAgent
from data_processing import preprocess_all_data, load_preprocessed_dataset, get_activity, get_column_units
from visualize import visualize_results, visualize_training, visualize_preprocessed_data
import numpy as np
from sklearn.model_selection import train_test_split
import torch

# Load Data

In [10]:
# Load the preprocessed dataset

df = load_preprocessed_dataset()

# Split data into 80% training and 20% testing sets
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

discrete_columns = [
        'Activity ID', 'Age', 'Height', 'Weight', 'Resting HR', 'Max HR',
        'Sex - Female', 'Sex - Male', 'Dominant Hand - Left', 'Dominant Hand - Right'
]

continuous_columns = [
        'Timestamp', 'Heart Rate', 'Hand Sensor - Temperature',
        'Hand Sensor - Accelerometer - X', 'Hand Sensor - Accelerometer - Y',
        'Hand Sensor - Accelerometer - Z', 'Hand Sensor - Gyroscope - X',
        'Hand Sensor - Gyroscope - Y', 'Hand Sensor - Gyroscope - Z',
        'Hand Sensor - Magnetometer - X', 'Hand Sensor - Magnetometer - Y',
        'Hand Sensor - Magnetometer - Z', 'Chest Sensor - Temperature',
        'Chest Sensor - Accelerometer - X', 'Chest Sensor - Accelerometer - Y',
        'Chest Sensor - Accelerometer - Z', 'Chest Sensor - Gyroscope - X',
        'Chest Sensor - Gyroscope - Y', 'Chest Sensor - Gyroscope - Z',
        'Chest Sensor - Magnetometer - X', 'Chest Sensor - Magnetometer - Y',
        'Chest Sensor - Magnetometer - Z', 'Ankle Sensor - Temperature',
        'Ankle Sensor - Accelerometer - X', 'Ankle Sensor - Accelerometer - Y',
        'Ankle Sensor - Accelerometer - Z', 'Ankle Sensor - Gyroscope - X',
        'Ankle Sensor - Gyroscope - Y', 'Ankle Sensor - Gyroscope - Z',
        'Ankle Sensor - Magnetometer - X', 'Ankle Sensor - Magnetometer - Y',
        'Ankle Sensor - Magnetometer - Z'
]

print(df_train.columns)
print(df.head())

--- Preprocessed Dataset Info ---
Number of rows: 175498
Number of columns: 42
---------------------------------
Index(['Timestamp', 'Activity ID', 'Heart Rate', 'Hand Sensor - Temperature',
       'Hand Sensor - Accelerometer - X', 'Hand Sensor - Accelerometer - Y',
       'Hand Sensor - Accelerometer - Z', 'Hand Sensor - Gyroscope - X',
       'Hand Sensor - Gyroscope - Y', 'Hand Sensor - Gyroscope - Z',
       'Hand Sensor - Magnetometer - X', 'Hand Sensor - Magnetometer - Y',
       'Hand Sensor - Magnetometer - Z', 'Chest Sensor - Temperature',
       'Chest Sensor - Accelerometer - X', 'Chest Sensor - Accelerometer - Y',
       'Chest Sensor - Accelerometer - Z', 'Chest Sensor - Gyroscope - X',
       'Chest Sensor - Gyroscope - Y', 'Chest Sensor - Gyroscope - Z',
       'Chest Sensor - Magnetometer - X', 'Chest Sensor - Magnetometer - Y',
       'Chest Sensor - Magnetometer - Z', 'Ankle Sensor - Temperature',
       'Ankle Sensor - Accelerometer - X', 'Ankle Sensor - Acceleromet

# Initialize system

In [11]:
#discrete action size columns
dqn_config = {
    'state_size': df.shape[1] - 1,  
    'action_size': len(discrete_columns),  
    'hidden_size': 64,
    'lr': 1e-3,
    'gamma': 0.99,
    'batch_size': 32,
    'memory_size': 10000,
    'epsilon_start': 1.0,
    'epsilon_min': 0.01,
    'epsilon_decay': 0.995
}


#continuous
ppo_config = {
    'state_size': df.shape[1] - 1,  
    'action_size': len(continuous_columns),   
    'hidden_size': 64,
    'lr': 3e-4,
    'gamma': 0.99,
    'clip_epsilon': 0.2,
    'update_epochs': 4,
    'batch_size': 64,
    'c1': 0.5,
    'c2': 0.01
}

ffnn_config = {
    'input_size': df.shape[1] - 1,
    'hidden_sizes': [64, 64],
    'output_size': 1,
    'learning_rate': 0.001,
    'batch_size': 32,
    'epochs': 1,
    'type': 'regression'
}


dqn_agent = DQNAgent(**dqn_config)
ppo_agent = PPOAgent(**ppo_config)
ffnn_agent = FFNNAgent(**ffnn_config)

Using device: cpu


# Training

In [14]:
#generate data (actions)
#neural net trains on data
#predict on normal dataset
#return rewards and accuracy to plot
def evaluate_ffnn(ffnn_agent, data, labels):
    # Ensure data is converted to NumPy array first
    if isinstance(data, np.ndarray) == False:
        data = data.to_numpy()
    if isinstance(labels, np.ndarray) == False:
        labels = labels.to_numpy()

    # Convert to PyTorch tensors
    data_tensor = torch.tensor(data, dtype=torch.float32)
    labels_tensor = torch.tensor(labels, dtype=torch.float32)

    device = next(ffnn_agent.model.parameters()).device
    data_tensor, labels_tensor = data_tensor.to(device), labels_tensor.to(device)

    predictions = ffnn_agent.predict(data_tensor)
    mse = torch.mean((torch.tensor(predictions) - labels_tensor) ** 2).item()
    accuracy = 1 - mse
    return -mse, accuracy

def train_agents(df, dqn_agent, ppo_agent, ffnn_agent, episodes=100, save_path='training_metrics.json'):
    SYNTHETIC_SAMPLES = int(0.1 * len(df))
    rewards = []
    baseline_rewards = []
    baseline_accuracies = []
    accuracies = []

    ffnn_agent.train(df_train.iloc[:, :-1].values, df_train.iloc[:, -1:].values)
    baseline_reward, baseline_accuracy = evaluate_ffnn(ffnn_agent, df_train.iloc[:, :-1], df_train.iloc[:, -1:])
    baseline_rewards.append(baseline_reward)
    baseline_accuracies.append(baseline_accuracy)
    print(f"Baseline | Reward: {baseline_reward:.4f} | Accuracy: {baseline_accuracy:.4f}")


    for episode in range(episodes):
        synthetic_data = []
        synthetic_labels = []

        for _ in range(SYNTHETIC_SAMPLES):
            state = df.sample(n=1).iloc[:, :-1].values.flatten()  # Example random state for RL
            discrete_action = dqn_agent.predict(state)  # Discrete variable data
            continuous_action = ppo_agent.predict(state)  # Continuous variable data

            print(f'length of discrete action: {discrete_action}')

            synthetic_sample = np.zeros((1, df.shape[1] - 1))
            synthetic_sample[:, df.columns.get_indexer(discrete_columns)] = discrete_action
            synthetic_sample[:, df.columns.get_indexer(continuous_columns)] = continuous_action

            synthetic_data.append(synthetic_sample)
            synthetic_labels.append(np.random.rand(1, 1))  # Example label logic

        synthetic_data = np.vstack(synthetic_data)
        synthetic_labels = np.vstack(synthetic_labels)

        # Train FFNN on synthetic data
        ffnn_agent.train(synthetic_data, synthetic_labels)

        # Evaluate FFNN on original data to compute reward and accuracy
        reward, accuracy = evaluate_ffnn(ffnn_agent, df.iloc[:, :-1], df.iloc[:, -1:])
        rewards.append(reward)
        accuracies.append(accuracy)

        # Store experience in RL agents and train them
        dqn_agent.remember(state, discrete_action, reward, state, False)
        ppo_agent.store_transition(state, continuous_action, 0, reward, False, 0)  # Adjust if needed
        dqn_agent.train()
        ppo_agent.train()

        print(f"Episode {episode + 1}/{episodes} | Reward: {reward:.4f} | Accuracy: {accuracy:.4f}")

    # Save metrics
    metrics = {
        'baseline_rewards': baseline_rewards,
        'baseline_accuracies': baseline_accuracies,
        'rewards': rewards,
        'accuracies': accuracies
    }
    import json
    with open(save_path, 'w') as f:
        json.dump(metrics, f)
    print(f"Metrics saved to {save_path}")

    return metrics


# Train agents and return results
results = train_agents(df_train, dqn_agent, ppo_agent, ffnn_agent)

#Show progressive results
print("Training DQN Agent...")
dqn_agent.save("dqn_trained_model.pth")

print("Training PPO Agent...")
ppo_agent.save("ppo_trained_model.pth")

Baseline | Reward: -0.0022 | Accuracy: 0.9978
length of discrete action: 3


IndexError: index 41 is out of bounds for axis 1 with size 41

# Testing


In [ ]:
def test_agent(ffnn_agent, test_df):
    test_data = test_df.iloc[:, :-1].values
    test_labels = test_df.iloc[:, -1:].values
    reward, accuracy = evaluate_ffnn(ffnn_agent, test_data, test_labels)
    print(f"Test Results | Reward: {reward:.4f} | Accuracy: {accuracy:.4f}")

# Test FFNN on testing set
test_agent(ffnn_agent, df_test)

# Visualize results

In [ ]:
# visualize_training([], [])
# visualize_results(dqn_agent, ppo_agent, df)
# visualize_preprocessed_data(df)